# Hourly Area Burned

This script takes cumulative hotspot detections calculated in GEE and creates a figure showing hourly fire growth by fire.

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import rasterio as rio
from rasterio.mask import mask
import matplotlib.pyplot as plt
import plotly

In [ ]:
TARGET_CRS = "EPSG:5070"

PIXEL_SIZE = 2500 * 2500
SQM_PER_HA = 10000

# Surface wind speed in meters per second from Cramer 1957
CRAMER_WIND_THRESHOLD = 4
CRAMER_RH_THRESHOLD = 36

## Load data

In [ ]:
mask_path = os.path.join("..", "data", "cumulative_hourly_masks.tif")
masks = rio.open(mask_path)

In [ ]:
fires_path = os.path.join("..", "data", "NIFC", "Public_NIFS_Perimeters_westside_ecoregions_complete.shp")

fires = gpd.read_file(fires_path).to_crs(TARGET_CRS)
# rasterio refuses to mask with a Polygon, so cast them all to Multipolygon
fires["geometry"] = [MultiPolygon([feature]) if type(feature) == Polygon else feature for feature in fires["geometry"]]

## Processing

### All fires

In [ ]:
raw_df = pd.DataFrame()

# Calculate the cumulative area in each fire at each hour and the corresponding weather
for i, fire in fires.iterrows():
    # Convert fire name to camel case
    name = fire.IncidentNa.title()
    geom = fire.geometry

    # Clip the hourly masks to the fire
    fire_masks, _ = mask(masks, geom, crop=False, nodata=0, filled=True, pad=False, all_touched=False)
    # Replace NaN with 0 (not sure if this is needed)
    fire_masks = np.nan_to_num(fire_masks)

    # Extract the fire mask in each hour
    for i, band in enumerate(masks.descriptions):
        # GEE adds an index to each band, so remove those
        date_str = ('_').join(band.split('_')[1:])

        # Convert the date string to a datetime object
        date = pd.to_datetime(date_str, format="%Y_%m_%d_%H")

        # Binary mask of all patches in one fire in one hour
        hour_mask = fire_masks[i].astype(bool)

        area_burned = np.count_nonzero(hour_mask) * PIXEL_SIZE / SQM_PER_HA

        # If no burning patches exist, skip this hour
        if not area_burned:
            continue

        row = {}
        row["fire_name"] = name
        row["date"] = date
        row["hectares"] = area_burned

        raw_df = raw_df.append(row, ignore_index=True)

## Figures

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

### Cumulative fire area

In [ ]:
df = raw_df.copy()

df["prev_hectares"] = df.groupby("fire_name").hectares.shift(1)
# Calculate the area grown in the last hour
df["hectare_growth"] = df.hectares.subtract(df.prev_hectares)


# Subtract 1 hour. The mask bands represent end dates, so this shifts them to start dates.
df.date = df.date - pd.Timedelta("1 hour")

# Remove the first hour of the 14th to get 240 hours
df = df[df.date.lt("2020-09-14")]


# Remove S. Obenchain since it's too far south
df = df[df.fire_name.ne("S. Obenchain")]

# Lump all minor fires into "Other"
major_fires = ["Big Hollow", "Archie Creek", "Riverside", "Holiday Farm", "Lionshead", "Beachie Creek"]
df.fire_name = df.fire_name.apply(lambda x : x if x in major_fires else "Other")

# Re-group and sum by hour to aggregate the "other" fires
df = df.groupby(["fire_name", "date"]).sum().reset_index()

# Calculate the final size of each fire
df = df.assign(final_hectares=df.groupby("fire_name").hectares.transform(max))
# Sort by largest fire
df = df.sort_values(by=["final_hectares", "date"], ascending=False)

In [ ]:
# Hectares burned during the 48 hour window
df_48hr = df[df.date.ge("2020-09-08T00") & df.date.lt("2020-09-10T00")]
df_48hr.groupby("fire_name").hectare_growth.sum().sum()

In [ ]:
fill_colors = px.colors.qualitative.Vivid[1:]

fig = px.area(df, x="date", y="hectares", color="fire_name", 
              color_discrete_sequence=fill_colors,
             labels={"fire_name": ""})

fig.update_layout(
    template="ggplot2",
    yaxis_title="Cumulative Hectares Burned",
    xaxis_title="Date",
    plot_bgcolor='rgb(255, 255, 255)',
    # Format the date labels
    xaxis=dict(
        tickmode = "linear",
        tickformat="%m/%d",
        ticklabelmode="period"
    ),
    # Flip the order of the legend to match the order of the areas
    legend=dict(
        traceorder="reversed"
    )
)

# Add solid plot border and grey gridlines
fig.update_yaxes(showline=True, linecolor="black", linewidth=1, mirror=True, gridcolor="rgba(0, 0, 0, 0.1)")
fig.update_xaxes(showline=True, linecolor="black", linewidth=1, mirror=True, gridcolor="rgba(0, 0, 0, 0.1)")

# Highlight 48 hour period
fig.add_vrect(x0="2020-09-08T00", x1="2020-09-10T00", 
                      fillcolor="red", opacity=0.2, line_width=0, layer="below")

# Highlight 48 hour period
fig.add_vrect(x0="2020-09-08T00", x1="2020-09-10T00", 
              fillcolor="rgba(0, 0, 0, 0)", opacity=1, 
              line_width=1, line_color="red", layer="above", line_dash="dot")

# Manually assign the colors defined above so that I can override the default alpha
for i, trace in enumerate(fig['data']):
    trace['fillcolor'] = fill_colors[i]
    trace['line_color'] = "rgba(0, 0, 0, 1)"
    trace['line_width'] = 0.5
   
fig.update_layout(
    width=800,
    height=360
)

fig

In [ ]:
# Area that burned in a 48 hour period
period_burned = df[df.date.gt("2020-09-08T00") & df.date.le("2020-09-10T00")].groupby("date").hectare_growth.sum().sum(0)

In [ ]:
# Total cumulative area burned by the end of the time range.
total_burned = df.groupby("date").sum().reset_index().hectares[-1:]

# Final fire size for these fires in 2020, according to GIS
final_burned = 379045.165

In [ ]:
period_burned